In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import IntegerType

import os

In [2]:
# change to be reflective of your environment
data_dir = '/Users/zhaoluyang/Downloads/Senior-Capstone-2018-2019-master/Notebooks/TestData'

In [3]:
import os
size = os.path.getsize(data_dir+"/BACKUP_OBJECTS.csv")
end = 50 # how many gigs to scale backup objects
upscale = int((end * 1073741824) / size)
upscale

13

In [4]:
# initiates a SparkContext which is necessary for accessing data in Spark
sc = SparkContext()
sqlContext = SQLContext(sc)
# change to match your environment
output_dir = data_dir + "/merge_data"

In [5]:
SDRO = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir + '/SD_RECON_ORDER.csv'])
SS_POOLS = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir + '/SS_POOLS.csv'])
AFBF = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/AF_BITFILES.csv"])
BACKUP_OBJECTS = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/BACKUP_OBJECTS.csv"])
ls = [SDRO, SS_POOLS, AFBF, BACKUP_OBJECTS]

In [12]:
# upscale step
for i in range(len(ls)):
    csv = ls[i]
    concat = csv
    print(concat.count())
    for j in range(upscale):
        concat = concat.union(csv)
    ls[i] = concat
    print(concat.count())
    
    
SDRO = ls[0]
SS_POOLS = ls[1]
AFBF = ls[2]
BACKUP_OBJECTS = ls[3]

# # save new ones before join then reload
# SDRO.write.options(header='true').format('com.databricks.spark.csv').save(data_dir + "/upscale_" + str(end) + "_SDRO")
# SS_POOLS.write.options(header='true').format('com.databricks.spark.csv').save(data_dir + "/upscale_" + str(end) + "_SS_POOLS")
# AFBF.write.options(header='true').format('com.databricks.spark.csv').save(data_dir + "/upscale_" + str(end) + "_AFBF")
# BACKUP_OBJECTS.write.options(header='true').format('com.databricks.spark.csv').save(data_dir + "/upscale_" + str(end) + "_BACKUP_OBJECTS")


53161928


KeyboardInterrupt: 

In [ ]:
# SDRO = None
# SS_POOLS = None
# AFBF = None
# BACKUP_OBJECTS = None

In [ ]:
# SDRO = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load(data_dir + "/upscale_" + str(end) + "_SDRO/*.csv")
# SS_POOLS = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load(data_dir + "/upscale_" + str(end) + "_SS_POOLS/*.csv")
# AFBF = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load(data_dir + "/upscale_" + str(end) + "_AFBF/*.csv")
# BACKUP_OBJECTS = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load(data_dir + "/upscale_" + str(end) + "_BACKUP_OBJECTS/*.csv")

In [ ]:
full_outer_join = BACKUP_OBJECTS.join(AFBF, BACKUP_OBJECTS.OBJID == AFBF.BFID,how='left') # Could also use 'full_outer'
full_outer_join = full_outer_join.join(SDRO, ['OBJID'],how='left') # Could also use 'full_outer'

In [ ]:
SDRO = None
SS_POOLS = None
AFBF = None
BACKUP_OBJECTS = None

In [ ]:
df = full_outer_join
df = df.filter(df.POOLID. isNotNull())
df = df.withColumn("POOLID", df["POOLID"].cast("int"))
df = df.filter(df.ATTRLENGTH. isNotNull())
df = df.withColumn("ATTRLENGTH", df["ATTRLENGTH"].cast("float"))
df = df.filter(df.BFSIZE. isNotNull())
df = df.withColumn("BFSIZE", df["BFSIZE"].cast("float"))
df = df.filter(df.HDRSIZE. isNotNull())
df = df.withColumn("HDRSIZE", df["HDRSIZE"].cast("float"))
df = df.filter(df.OBJID. isNotNull())
df = df.withColumn("OBJID", df["OBJID"].cast("float"))

In [ ]:
from pyspark.sql.functions import when   

df = df.withColumn('POOLID', when(df['POOLID'] == -1000000, 0).otherwise(df['POOLID']))
df = df.withColumn('POOLID', when(df['POOLID'] == -9, 1).otherwise(df['POOLID']))
df = df.withColumn('POOLID', when(df['POOLID'] == 4, 2).otherwise(df['POOLID']))
df = df.withColumn('POOLID', when(df['POOLID'] == 6, 3).otherwise(df['POOLID']))
df = df.withColumn('POOLID', when(df['POOLID'] == 42, 4).otherwise(df['POOLID']))
df = df.withColumn('POOLID', when(df['POOLID'] == 72, 5).otherwise(df['POOLID']))
df = df.withColumn('POOLID', when(df['POOLID'] == 82, 6).otherwise(df['POOLID']))
df = df.withColumn('POOLID', when(df['POOLID'] == -1, 7).otherwise(df['POOLID']))

In [ ]:
df = df.filter(df.POOLID. isNotNull())
df = df.withColumn("POOLID", df["POOLID"].cast("int"))
df = df.filter(df.ATTRLENGTH. isNotNull())
df = df.withColumn("ATTRLENGTH", df["ATTRLENGTH"].cast("float"))
df = df.filter(df.BFSIZE. isNotNull())
df = df.withColumn("BFSIZE", df["BFSIZE"].cast("float"))
df = df.filter(df.HDRSIZE. isNotNull())
df = df.withColumn("HDRSIZE", df["HDRSIZE"].cast("float"))
df = df.filter(df.OBJID. isNotNull())
df = df.withColumn("OBJID", df["OBJID"].cast("float"))

In [16]:
df.select("OBJID", "ATTRLENGTH", "BFSIZE", "HDRSIZE", "POOLID").write.options(header='true').format('com.databricks.spark.csv').save(data_dir + "/merge_data/upscale")
                                                                                                    
                                                                                                                                                   
                                                                                                    

Py4JJavaError: An error occurred while calling o327.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:196)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.sc

In [ ]:
df = None

In [ ]:
sc.stop()

In [1]:
# Dependencies
import glob, os
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
# Data preprocess
def data_preprocess(next):

    tf_features = [next['OBJID'], next['ATTRLENGTH'], next['BFSIZE'], next['HDRSIZE']]
    features_columns = np.array(sess.run(tf_features), dtype='float32')

    #Normalization
    for column in range(4):
        column_range = features_columns[column].max() - features_columns[column].min()
        if column_range != 0.0: 
            features_columns[column] = (features_columns[column] - features_columns[column].min())/column_range
    # print(features_columns)

    features = np.array([]).reshape(0,4)
    for i in range(batch_len):
        ele = np.zeros(4)
        np.put(ele,0,features_columns[0][i])
        np.put(ele,1,features_columns[1][i])
        np.put(ele,2,features_columns[2][i])
        np.put(ele,3,features_columns[3][i])
        features = np.r_[features, [ele]]
    # print(features)
    
    
    
    tf_labels = next['POOLID']
    labels = np.array(sess.run(tf_labels), dtype='float32')
#     print(labels)

    # One-hot encoding for the categories
    num_classes = 8
    targets = np.array([]).reshape(0,num_classes)

    for i in range(0,batch_len):
        ele = np.zeros(num_classes)
        np.put(ele,labels[i],1)
        targets = np.r_[targets, [ele]]
    
#     return features, targets
    
#     Shuffle Data
    indices = np.random.choice(len(features), len(features), replace=False)
    X_data = features[indices]
    y_data = targets[indices]
    
    X_values = features[indices]
    y_values = targets[indices]
    
    return X_data, y_data

In [3]:
def training(X_train, y_train, start_epoch):
    
    # Interval / Epochs
    interval = 500
    epoch = 1500

    # Training the model...
    for i in range(1, (epoch + 1)):
        sess.run(optimizer, feed_dict={X_data: X_train, y_target: y_train})
        if i % interval == 0:
            print('Epoch', i + start_epoch, '|', 'Loss:', sess.run(loss, feed_dict={X_data: X_train, y_target: y_train}))
            
    return i + start_epoch


In [4]:
# get the accuracy of the model
def predict(X_test, y_test):
    correct_prediction = tf.equal(tf.argmax(final_output, 1), tf.argmax(y_target,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("The accuracy of the model is:", sess.run(accuracy, feed_dict={X_data: X_test, y_target: y_test}))

In [44]:
# define a neural network

# Initialize placeholders
X_data = tf.placeholder(shape=[None, 4], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, 8], dtype=tf.float32)

#create seed for random_normal()
seed = 1234
np.random.seed(seed)
tf.set_random_seed(seed)
output_nodes = 8
hidden_layer_nodes = 10

# We create a neural Network which contains 3 layers with 4, 10, 5 nodes repectively
w1 = tf.Variable(tf.random_normal(shape=[4,hidden_layer_nodes])) # Weight of the input layer
b1 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes]))   # Bias of the input layer
w2 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes,output_nodes])) # Weight of the hidden layer
b2 = tf.Variable(tf.random_normal(shape=[output_nodes]))                    # Bias of the hidden layer
hidden_output = tf.nn.relu(tf.add(tf.matmul(X_data, w1), b1))
final_output = tf.nn.softmax(tf.add(tf.matmul(hidden_output, w2), b2))

# Loss Function
loss = tf.reduce_mean(-tf.reduce_sum(y_target * tf.log(final_output), axis=0))

# Optimizer
# optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(loss)
# optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate=0.001).minimize(loss)
# optimizer = tf.train.ProximalAdagradOptimizer(learning_rate=0.01).minimize(loss)
# optimizer = tf.train.FtrlOptimizer(learning_rate=0.01).minimize(loss)
optimizer = tf.train.AdagradOptimizer(learning_rate=0.01).minimize(loss)
# optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
# optimizer = tf.train.AdadeltaOptimizer(learning_rate=0.01).minimize(loss)
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)

print("A neural Network which contains 3 layers with 4, 10, 8 nodes repectively was created!")

A neural Network which contains 3 layers with 4, 10, 8 nodes repectively was created!


In [45]:
path = '/Users/zhaoluyang/Downloads/Senior-Capstone-2018-2019-master/Notebooks/TestData/merge_data/upscale'                     
# path = data_dir + "/merge_data/upscale"
all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent

df_from_each_file = (pd.read_csv(f) for f in all_files)
concatenated_dataset   = pd.concat(df_from_each_file, ignore_index=True)
# print(len(concatenated_dataset))

batch_len = 4000
dataset = tf.data.experimental.make_csv_dataset(all_files, batch_size=batch_len)
# print(dataset)

start_epoch = 0
print('Training the model...')
iter = dataset.make_one_shot_iterator()

# Initialize variables
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

#There are total 2,157,764 rows in the combined csv files

#choose first 4000 rows as test data
next = iter.get_next()
X_test, y_test = data_preprocess(next)

#then 538*4000 = 2,152,000 rows will be used as training data
#538
for i in range(10):
    next = iter.get_next()
    # next is a dict with key=columns names and value=column data
    X_train, y_train = data_preprocess(next)
#     print(X_train)
    start_epoch = training(X_train, y_train, start_epoch)
    
predict(X_test, y_test)
    
print("Training finished\n")

Training the model...
Epoch 500 | Loss: 746.71234
Epoch 1000 | Loss: 556.2732
Epoch 1500 | Loss: 519.7567
Epoch 2000 | Loss: 612.8431
Epoch 2500 | Loss: 605.0314
Epoch 3000 | Loss: 599.23584
Epoch 3500 | Loss: 614.42773
Epoch 4000 | Loss: 607.87354
Epoch 4500 | Loss: 603.09033
Epoch 5000 | Loss: 467.77026
Epoch 5500 | Loss: 464.4257
Epoch 6000 | Loss: 462.08212
Epoch 6500 | Loss: 459.17407
Epoch 7000 | Loss: 456.2758
Epoch 7500 | Loss: 454.05948
Epoch 8000 | Loss: 512.5681
Epoch 8500 | Loss: 508.61023
Epoch 9000 | Loss: 507.40106
Epoch 9500 | Loss: 531.83655
Epoch 10000 | Loss: 529.4994
Epoch 10500 | Loss: 528.27075
Epoch 11000 | Loss: 512.43787
Epoch 11500 | Loss: 511.53534
Epoch 12000 | Loss: 510.8705
Epoch 12500 | Loss: 494.6507
Epoch 13000 | Loss: 494.0233
Epoch 13500 | Loss: 493.59396
Epoch 14000 | Loss: 591.53735
Epoch 14500 | Loss: 590.6937
Epoch 15000 | Loss: 590.2887
The accuracy of the model is: 0.703
Training finished



In [ ]:
# Prediction
np.set_printoptions(precision=4)
unknown = np.array([[0.693363, 0.0, 0.002894, 0.148097]], dtype=np.float32)
predicted = sess.run(final_output, feed_dict={X_data: unknown})
# model.predict(unknown)
print("Using model to predict pool id for features: ", unknown)
print("\nPredicted softmax vector is: ",predicted)
Class_dict={'POOLID_-1000000': 0, 'POOLID_-9': 1, 'POOLID_-1': 2, 'POOLID_4': 3, 'POOLID_-1': 4, 'POOLID_6': 5, 'POOLID_42': 6, 'POOLID_72': 7, 'POOLID_82': 8 }
pool_dict = {v:k for k,v in Class_dict.items()}
print("\nPredicted pool id is: ", pool_dict[np.argmax(predicted)])